In [540]:
import numpy as np
import pandas as pd
import re

In [541]:
record = []
with open('Day4.txt') as f:
    for line in f.readlines():
        record_temp = line.split('] ')
        if '#' in record_temp[1]:
            notes = int(re.sub('[^0-9]','',record_temp[1]))
        else:
            notes = record_temp[1].strip()
            if notes == 'falls asleep':
                notes = 1
            elif notes == 'wakes up':
                notes = 0
        record.append([record_temp[0].strip('['),notes])
record_df = pd.DataFrame(record,columns=['Datetime','Notes'])
record_df[['Date','Time']] = record_df['Datetime'].str.split(expand=True)
record_df[['Year','Month','Day']] = record_df['Date'].str.split('-',expand=True)
record_df[['Hour','Minute']] = record_df['Time'].str.split(':',expand=True)
record_df['Datetime_New'] = pd.to_datetime(record_df['Month'] + '-' + record_df['Day'] + '-' + '2018' 
               + ' ' + record_df['Hour'] + ':' + record_df['Minute'])
record_df.set_index('Datetime_New',inplace=True)
record_df.sort_index(inplace=True)
record_df = record_df[['Month','Day','Hour','Minute','Notes']].reset_index(drop=True)
record_df[['Month','Day','Hour','Minute']] = record_df[['Month','Day','Hour','Minute']].astype(int)

#Match Guard ID Number to Date
guard_date = record_df.loc[record_df['Notes'] > 3,['Month','Day','Hour','Notes']]
def add1(row):
    if row['Hour'] != 0:
        row['Day'] +=1
    return row['Day']
guard_date['Day'] = guard_date.apply(add1,axis=1)

uni_date = record_df.loc[record_df['Hour'] == 0,
                         ['Month','Day','Hour']].drop_duplicates(['Month','Day']).reset_index(drop=True)
min60 = pd.DataFrame(np.transpose(np.vstack((np.zeros(60,dtype=int),np.arange(0,60)))),columns=['Hour','Minute'])
table = uni_date.merge(min60,on ='Hour').merge(guard_date[['Month','Day','Notes']],on=['Month','Day'])
table.rename(columns={'Notes':'Guard ID'},inplace=True)

record_df.loc[record_df['Notes']>3,'Notes'] = 0
table = table.merge(record_df.loc[record_df['Hour'] == 0,['Month','Day','Hour','Minute','Notes']],
            on=['Month','Day','Hour','Minute'],how='left')
table.set_index(['Month','Day'],inplace=True)

In [554]:
def spread(array):
    #Save first rep. 
    rep = np.nan
    rep1 = np.nan
    for i in range(0,60):
        if ~np.isnan(array[i]) & np.isnan(rep1):
            rep = array[i].copy()
            rep1 = rep
        elif ~np.isnan(array[i]):
            rep = array[i].copy()
        elif ~np.isnan(rep):
            array[i] = rep
    if rep1 == 1:
        array[np.isnan(array)] = 0
    return array

for index in table.index.unique():
    table_temp = spread(table.loc[index,'Notes'].to_numpy())
    table.loc[index,'Notes'] = table_temp
    
table_pivot = table.pivot_table(index=['Month','Day','Guard ID'],
            columns='Minute',values='Notes',aggfunc=np.sum)
guard_max = table_pivot.groupby('Guard ID').sum().sum(axis=1).sort_values(ascending=False).idxmax() #Guard 1901

In [555]:
table_grouped = table_pivot.groupby('Guard ID')
minute_max = table_grouped.get_group(1901).sum(axis=0).idxmax()
guard_max*minute_max

77941

In [570]:
#Part 2
minute_max2 = table_pivot.groupby('Guard ID').sum().max().idxmax()
guard_max2 = table_pivot.groupby('Guard ID').sum().idxmax()[27]
guard_max2*minute_max2

35289